# Relational Classes

## The Relational Database

In this case we had a variable (<code>DbPath</code>) - containing the path of the database - and  actually two methods to implement 
* <code>getDbPath</code> to retrieve the path of the database, which returns a string 
* <code>setDbPath</code> to instead set up a new database path, which takes in input a string - the path - and returns a boolean (True in the case the path passed is empty, False if it isn't).

In [ ]:
class RelationalProcessor(object): 
    def __init__(self, dbPath=None):
        self.dbPath=dbPath  
    def getDbPath(self):
        return self.dbPath
    def setDbPath(self, path):
        if path!='':
            self.dpPath=path
            return True
        else:
            return False

We created these classes and methods basing ourself on the [data model](https://github.com/comp-data/2021-2022/tree/main/docs/project), which had been procured to us and basing ourselves on the lessons we attended.

We defined the name of the class and then worked onto writing the method and most importantly the constructor, which will be called everytime a new object is built.

## The Relational Data Processor

Its single method is <code>uploadData</code>, which was used to insert the collection of data of the input path specified inside the database

In this case we worked with two main data formats, which were CSV and JSON.

* The first step in this case, was understanding the structure of these two different formats and of the data contained inside, always paying attention to the data model.
* Once understood the structure, we moved onto understanding what information we could retrieve from the JSON and which from the CSV.
* First of all we checked with an <i>if</i> whether the path in the input was empty or not.
* Then we created another <i>if</i> node to check whether the data format passed was a CSV or a JSON, through the method <code>[endswith](https://www.w3schools.com/python/ref_string_endswith.asp)</code>.
* Once we separated the data formats, we worked onto producing the different <code>DataFrames</code> that we'd need for the database, from the data formats. 
Since we'd be dealing with a relational database, in this case we worked with <q>a relational model of data, using tables of columns and row where each column is identified by a unique key</q>.
* Through the method <code>[read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)</code> we created a dataframe out of the initial CSV and then worked onto cleaning it and renaming a few columns, ending up with a table (<i>Publications</i>) that had the following columns:
    * <b>publicationInternalId</b>, which worked as an <i>InternalId</i> and was then extremely useful later for the questions.
    * <b>doi</b> of the publication.
    * <b>title</b>.
    * <b>type</b>.
    * <b>type</b>.
    * <b>publicationYear</b>.
    * <b>issue</b>, which characterized the subclass of <i>Journal Article</i>.
    * <b>volume</b>, which characterized the subclass of <i>Journal Article</i>.
    * <b>chapterNumber</b>, which characterized the subclass of <i>BookChapter</i>.
    * <b>publicationVenue </b>, in this case not the relationship but the name of the venue, in which the publication was published.
    * <b>venue_type</b>.
    * <b>publisher</b>.
    * <b>event</b>, which characterized the subclass of <i>Proceedings</i>.
* After this we used the DataFrame method <i>to_sql</i> to add our table in the database.
* But before this, we employed [SQLite](https://www.sqlite.org) to operate on the database, using the class <code>Connection</code>, to connect to a particular database, stored in a <i>db</i> file.
This was obtained through the function <code>connect</code>.

In [ ]:
class RelationalDataProcessor (RelationalProcessor):
    def __init__(self, dbPath=None):
        super().__init__(dbPath)
    def uploadData (self, path):
        import pandas as pd
        from pandas import DataFrame
        from sqlite3 import connect
        from pandas import Series
        if path !='':
            if path.endswith(".csv"):
                from pandas import read_csv
            
                df_rel_publications=read_csv (path,
                                    keep_default_na=False,
                                    encoding="utf-8",
                                    dtype={
                                "id": "string",
                                "title": "string",
                                "type": "string",
                                "publication_year": "int",
                                "issue": "string",
                                "volume": "string",
                                "chapter": "string",
                                "publication_venue": "string",
                                "venue_type": "string",
                                "publisher":"string",
                                "event":"string"            
                                }) 
                # Create a new column with internal identifiers for each publication
                publication_internal_id = []
                for idx, row in df_rel_publications.iterrows():
                    publication_internal_id.append("publication-" + str(row["id"]))
                df_rel_publications.insert(0, "publicationInternalId", Series(publication_internal_id, dtype="string"))
                publisher_internal_id = []
                for idx, row in df_rel_publications.iterrows():
                    publisher_internal_id.append ("publisher-" + str(row["publisher"]))
                del df_rel_publications["publisher"]
                df_rel_publications.insert(10, "publisher", Series(publisher_internal_id, dtype="string"))

                df_rel_publications= df_rel_publications.rename(columns={"id":"doi", "publication_year":"publicationYear", 
                "chapter":"chapterNumber", "publication_venue":"publicationVenue"})

                with connect(self.dbPath) as con:
                    df_rel_publications.to_sql("Publications", con, if_exists="replace", index=False)
                    con.commit()
                return True

We then worked onto the data contained in the JSON file.

* In this case we didn't immediately turn the JSON in a <code>DataFrame</code>, but instead after having loaded, we worked through accessing all its various sections, and only then we worked onto turning it in a <code>DataFrame</code>, through <code>[pd.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)</code>.
* To further access to the data we needed we also used the <code>[iloc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html)</code> property to select the positions we needed, in the <code>DataFrames</code>.
* Looking at the data, contained in the JSON, we soon realized that it was possible that the same publication might have more than one venue, author and citation, so to obviate to this problem, we created a single string out of the lists of venues and citations present inside of the JSON, since the lists themselves were unsupported by SQLite.
* In this case we had 4 four main sections:
    * <b>Authors</b>, which had the following columns:
        * <b>authorInternalId</b>, created for the queries in the next step and containing the InternalId of the authors.
        * <b>familyName</b>
        * <b>givenName</b>
        * <b> authorID</b>, the orcid.
        * <b>publicationInternalId</b>, which would be later used as a key element for the queries.
    * <b>Venues</b>, which had the following columns:
        * <b>venueInternalId</b>, created for the queries in the next step and containing the InternalId of the venues.
        * <b>venueID </b>, the venues ids.
        * <b>publicationInternalId</b>, which would be later used as a key element for the queries.
    * <b>Publishers</b>, which had the following columns:
        * <b>publisherInternalId</b>, created for the queries in the next step and containing the InternalId of the venues.
        * <b>organizationID</b>, the ids of the publishers.
        * <b>name</b>, the names of the publishers.
    * <b>Cited publications</b>.
    In this case we created two tables, containing the same citations but in different ways:
        * <b>CitedPublications</b> contained the rows:
            * <b>id_all_references</b>, where all the citations were collected together in a single string, to avoid creating further problems in the generic stage.
            * <b> publicationInternalId</b>, created for the queries in the next step and containing the InternalId of the venues.
        * <b>CitedPublications1</b>, instead contained:
            * <b>doi</b> of the publication.
            * <b>CitedPublications</b>, with all the citations contained inside, but this time not linked together through a string, but instead for each different citation the doi is repeated inside the table, which created more rows, since some doi values were repeated.
* After all the dataframes were created we ended up uploading them as well in the database, through the same methods as described above.


In [ ]:
class RelationalDataProcessor (RelationalProcessor):
    def __init__(self, dbPath=None):
        super().__init__(dbPath)
    def uploadData (self, path):
        import pandas as pd
        from pandas import DataFrame
        from sqlite3 import connect
        from pandas import Series
        if path !='':
            if path.endswith(".csv"):
                from pandas import read_csv
            
                df_rel_publications=read_csv (path,
                                    keep_default_na=False,
                                    encoding="utf-8",
                                    dtype={
                                "id": "string",
                                "title": "string",
                                "type": "string",
                                "publication_year": "int",
                                "issue": "string",
                                "volume": "string",
                                "chapter": "string",
                                "publication_venue": "string",
                                "venue_type": "string",
                                "publisher":"string",
                                "event":"string"            
                                }) 
                # Create a new column with internal identifiers for each publication
                publication_internal_id = []
                for idx, row in df_rel_publications.iterrows():
                    publication_internal_id.append("publication-" + str(row["id"]))
                df_rel_publications.insert(0, "publicationInternalId", Series(publication_internal_id, dtype="string"))
                publisher_internal_id = []
                for idx, row in df_rel_publications.iterrows():
                    publisher_internal_id.append ("publisher-" + str(row["publisher"]))
                del df_rel_publications["publisher"]
                df_rel_publications.insert(10, "publisher", Series(publisher_internal_id, dtype="string"))

                df_rel_publications= df_rel_publications.rename(columns={"id":"doi", "publication_year":"publicationYear", 
                "chapter":"chapterNumber", "publication_venue":"publicationVenue"})

                with connect(self.dbPath) as con:
                    df_rel_publications.to_sql("Publications", con, if_exists="replace", index=False)
                    con.commit()
                return True

            if path.endswith(".json"):
                from json import load 
                with open(path, "r", encoding="utf-8") as f:
                    rel_data = load(f)

                #authors' dataframe
                findex=0
                for x in rel_data["authors"]:
                    for y in rel_data["authors"][x]:
                        findex+=1

                df_author=pd.DataFrame(columns=["doi","family","given","orcid"],index=range(findex))#dataframe with authors information
                ind=0
                for x in rel_data["authors"]:
                    for y in rel_data["authors"][x]:
                        df_author.iloc[ind] = (x,y["family"],y["given"],y["orcid"])
                        ind+=1
                publication_internal_id = []
                for idx, row in df_author.iterrows():
                    publication_internal_id.append("publication-" + str(row["doi"]))
                df_author.insert(4, "publicationInternalId", Series(publication_internal_id, dtype="string"))
                author_internal_id = []
                for idx, row in df_author.iterrows():
                    author_internal_id.append ("author-" + str(row["orcid"]))
                df_author.insert(0, "authorInternalId", Series(author_internal_id, dtype = "string"))
                    
                df_author = df_author[["authorInternalId", "family", "given", "orcid", "publicationInternalId"]]
                df_author= df_author.rename(columns={"given":"givenName", "family":"familyName","orcid":"authorID"})

                #dataframe with publications'doi and venues id
                findex = 0
                for x in rel_data["venues_id"]:
                    findex += 1
                df_pub_venues = pd.DataFrame(columns = ["doi","venues_id"], index= range(findex))
                ind =0
                for x in rel_data["venues_id"]:
                    df_pub_venues.iloc[ind] = (x, str(rel_data["venues_id"][x]))
                    ind +=1
                publication_internal_id =[]
                for idx, row in df_pub_venues.iterrows():
                    publication_internal_id.append("publication-" + str(row["doi"]))
                df_pub_venues.insert(0, "publicationInternalId", Series(publication_internal_id, dtype="string"))
                venue_internal_id= []
                for idx, row in df_pub_venues.iterrows():
                    venue_internal_id.append("venue-" + str(idx))
                df_pub_venues.insert(0, "venueInternalId", Series(venue_internal_id, dtype="string"))

                df_pub_venues = df_pub_venues[["venueInternalId", "venues_id", "publicationInternalId"]]
                df_pub_venues=df_pub_venues.rename(columns={"venues_id":"venueID"})

                #dataframe for publishers 
                findex = 0
                for x in rel_data["publishers"]:
                    findex+=1
                df_publishers = pd.DataFrame(columns=["publisher_id", "name"], index=range(findex))
                ind=0 
                for x in rel_data["publishers"]:
                    df_publishers.iloc[ind] = (rel_data["publishers"][x]["id"], rel_data["publishers"][x]["name"])
                    ind +=1
                publisher_internal_id = []
                for idx, row in df_publishers.iterrows():
                    publisher_internal_id.append ("publisher-" + str(row["publisher_id"]))
                df_publishers.insert(0, "publisherInternalId", Series(publisher_internal_id, dtype = "string"))
                df_publishers=df_publishers.rename(columns={"publisher_id":"organizationID"})
                
                findex_references=0
                for x in rel_data["references"]:
                    for y in rel_data["references"][x]:
                        findex_references +=1
                cited_publications = pd.DataFrame(columns=["id", "id_references"], index=range(findex_references))
                ind = 0
                for x in rel_data["references"]:
                    for y in rel_data["references"][x]:
                        cited_publications.iloc[ind]= (x, y)
                        ind += 1
                cited_publications = cited_publications.rename(columns={"id":"doi","id_references":"CitedPublications"})
            
                #references 
                findex=0
                for x in rel_data["references"]:
                    findex+=1
                df_references = pd.DataFrame(columns=["id", "id_references"], index=range(findex))
                ind = 0
                for x in rel_data["references"]:
                    df_references.iloc[ind]= (x, str(rel_data["references"][x]))
                    ind += 1        
                        
                for idx, row in df_references.iterrows():
                    publication_internal_id.append("publication-" + str(row["id"]))
                df_references.insert(2, "publicationInternalId", Series(publication_internal_id, dtype="string"))
                
                df_references = df_references[["id_references", "publicationInternalId"]]
                df_references=df_references.rename(columns={"id_references":"id_all_references"})

                # Cites
            
            
                with connect(self.dbPath) as con:
                    
                    df_author.to_sql("Authors", con, if_exists="replace", index=False)
                    df_publishers.to_sql("Publishers", con, if_exists="replace", index=False)
                    df_pub_venues.to_sql("Venues", con, if_exists="replace", index=False)
                    df_references.to_sql("CitedPublications", con, if_exists="replace", index=False)
                    cited_publications.to_sql("CitedPublications1", con, if_exists="replace", index=False)
                    con.commit()
                return True
            else:
                return False
        else:
            return False

## The Relational Query Processor

Now that we had effectively uploaded our data in our relational database, the next step was to retrieve the specific dataframes that we'd then need to actually implement the methods required.

To do so, we used the <b>[SQL](https://it.wikipedia.org/wiki/Structured_Query_Language)</b> (Structure Query Language) to further manage our database and retreieve the specific results we needed.

The first method that we had to implement was the <code>getPublicationsPublishedInYear</code>, which takes in input a number (the publication year, in this case) and returns a dataframe with all the publications published in that specific year.

In [ ]:
from pandas import merge 
from pandas import Series
from pandas import DataFrame
from sqlite3 import connect
from pandas import read_sql
from pandas import read_csv
from sqlite3 import connect

class RelationalQueryProcessor (RelationalProcessor):
    def __init__(self, dbpath):
        super().__init__(dbpath)


    def getPublicationsPublishedInYear(self, input_year):
        with connect(self.dbPath) as con:
            query = """SELECT Publications.publicationInternalId, Publications.doi, Publications.title, 
            Publications.publicationYear, Venues.venueID, CitedPublications.id_all_references, 
            GROUP_CONCAT(Authors.authorID)
            FROM Publications
            LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN CitedPublications ON CitedPublications.publicationInternalId == Publications.publicationInternalId
            WHERE Publications.publicationYear = '{0}'
            GROUP BY Publications.publicationInternalId;""".format(input_year)
            publication_year_df = read_sql(query, con)
            publication_year_df = publication_year_df.rename(columns={"GROUP_CONCAT(Authors.authorID)": "all_authors_id"})
        return publication_year_df

First of all, we connected to the database through the <code>with</code> clause but worked offline, to avoid any unexpected behaviors.

Following this we also needed the <code>[read_sql](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)</code> method, which enables us to use SQL on a SQL-based-database; it takes into consideration two parameters: 
* the query executed on the database (<code>query</code>).
* the connection to said database (<code>con</code>).

As for the part of the query, since we need ro retrieve a <b>Publication</b>, we chose in the <code>SELECT</code> statement <i>Publications.publicationInternalId</i>, <i>Publications.doi</i>, <i>Publications.title</i>, <i>Publications.publicationYear</i>, <i>Venues.venueID</i> (to express the relation <i>publicationVenue</i>), <i>CitedPublications.id_all_references</i>(to express the relation <i>cites</i>), <i>GROUP_CONCAT(Authors.authorID)</i>(to express the relation <i>author</i>), since they are the basic element characterizing such a class.

To retrieve all of these columns, we started <code>FROM</code> Publication and then through a series of <code>LEFT JOIN</code> we manage to join together the tables we need to fully return the <code>DataFrame</code> containing all the publication's essentials columns.

Then we come to the <code>WHERE</code> statement, where we specify the condition: in this case the Publications.publicationYear, needs to be same as the one stated by the input, which is obtained through the employment of the <code>[.format]("https://www.w3schools.com/python/ref_string_format.asp")</code> method.

This method formats the specificified value and insert it inside of the string's placeholder.

We used a similar method for the following questions which required to retrieve a <code>DataFrame</code> with all the publications, adapting our queries to the need of the method required.

For example in the case of the following queries, <code>getPublicationsByAuthorId</code>, have in the same rows, all the authors that collaborated to that publication, we created a second query, from which we retrieved a DataFrame with the columns, <i>GROUP_CONCAT(Authors.authorID)</i> and the <i>Publications.publicationInternalId</i>.

We then merged this <code>DataFrame</code> with the one we had obtained from the previous query, to make all the authors of the publication appear, instead of just the one we used in the input.

In [ ]:
def getPublicationsByAuthorId(self,authorID):
        with connect(self.dbPath) as con:
            query =  """SELECT Publications.publicationInternalId, Publications.doi, Publications.title, 
            Publications.publicationYear, Venues.venueID, CitedPublications.id_all_references
            FROM Publications
            LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN CitedPublications ON CitedPublications.publicationInternalId == Publications.publicationInternalId
            WHERE Authors.authorID = '{0}'
            GROUP BY Publications.publicationInternalId;""".format(authorID)
            publication_author_df = read_sql(query, con)
            
            query_2="""SELECT  GROUP_CONCAT(Authors.authorID), Publications.publicationInternalId
            FROM PUBLICATIONS
            LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
            GROUP BY Publications.publicationInternalId;"""
            all_authors= read_sql(query_2, con)
            all_authors= all_authors.rename(columns={"GROUP_CONCAT(Authors.authorID)":"all_authors_id"})
            
            publication_all_authors = publication_author_df.merge(all_authors)
            
        return publication_all_authors

We then had to work with the two methods <code>getMostCitedPublication</code> and <code>getMostCitedVenue</code>, which retrieve respectively:
* the DataFrame with all the publications that received the most number of citations.
* the Dataframe with all the venues containing the publications that received the most number of citations.

In this case we set to retrieve - for the first query - the same columns as we did for the previous questions, but in the FROM statement, we nestled another query, to set out to find inside of the CitedPublications1 DataFrame, which was the most cited publications.

To do so we used <code>COUNT</code> to count all the <i>CitedPublications1.citedPublications</i>, and store the name under the name <i>Cited</i> (which was added inside of the SELECT statement), we then grouped them all together through <code>GROUP BY</code> and then ordered them in a descendant order (<code>DESC</code>), setting the <code>LIMIT</code> at 1, this way we'd retrieve only the first result, which would be the one who had received the most citations.

After closing this nestled query, we went back to work on the original one, using the <code>LEFT JOIN</code> to retrieve all the columns which were missing from our starting point.

We had a similar process for the implementation for the getMostCitedVenue, but this time in the <code>SELECT</code> statement, we used all the columns that we associated to the Venue class (plus <i>Cited</i>):  <i>Venues.venueInternalId</i>, <i>Publications.publicationVenue</i>, <i>Venues.venueID</i>, <i>Publishers.organizationID</i>.

In [ ]:
def getMostCitedPublication(self):
    with connect(self.dbPath) as con:
        query ="""SELECT DISTINCT Publications.publicationInternalId, Publications.doi, Publications.title, 
        Publications.publicationYear, Venues.venueID, GROUP_CONCAT(Authors.authorID), Cited
        FROM (SELECT CitedPublications1."citedPublications", COUNT (CitedPublications1."citedPublications") as Cited
        FROM CitedPublications1
        GROUP BY CitedPublications1."citedPublications"
        ORDER BY Cited DESC
        LIMIT 1)
        LEFT JOIN Publications ON Publications.doi=="citedPublications"
        LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
        GROUP BY Publications.publicationInternalId;"""
        mostcitedpub=read_sql(query, con)
        mostcitedpub= mostcitedpub.rename(columns={"GROUP_CONCAT(Authors.authorID)": "all_authors_id"})
           
    return mostcitedpub
def getMostCitedVenue(self):
    with connect(self.dbPath) as con:
        query ="""SELECT DISTINCT Venues.venueInternalId, Publications.publicationVenue, Venues.venueID, Publishers.organizationID,  Cited
        FROM (SELECT CitedPublications1."citedPublications", COUNT (CitedPublications1."citedPublications") as Cited
        FROM CitedPublications1
        GROUP BY CitedPublications1."citedPublications"
        ORDER BY Cited DESC
        LIMIT 1)
        LEFT JOIN Publications ON Publications.doi=="citedPublications"
        LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN Publishers ON Publishers.publisherInternalId==Publications.publisher
        GROUP BY Publications.publicationInternalId;"""
        mostcitedvenue=read_sql(query, con)
        mostcitedvenue= mostcitedvenue.rename(columns={"GROUP_CONCAT(Authors.authorID)": "all_authors_id", "publicationVenue":"title"})
    return mostcitedvenue

Following these questions, we went back to the usual system, using the <code>SELECT</code> statement for the class Venue for the method <code>getVenuesbyPublisherId</code>, and the <code>LEFT JOIN</code> option, to obtain all the columns we needed to execute the queries.

We used the same logic for also the method <code>getPublicationInVenue</code>, but retrieving the columns for the class Publication.

In [ ]:
def getVenuesByPublisherId(self,publisherid):
        with connect(self.dbPath) as con:
            query="""SELECT Venues.venueInternalId, Publications.publicationVenue, Venues.venueID, Publishers.organizationID
            FROM Publications
            LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId 
            LEFT JOIN Publishers ON Publishers.publisherInternalId==Publications.publisher
            WHERE Publications.publisher LIKE '%{0}%'
            GROUP BY Publications.publicationInternalId;""".format(publisherid)
            venbypub=read_sql(query, con)
            venbypub=venbypub.rename(columns={"publicationVenue":"title"})
        return venbypub
def getPublicationInVenue(self,venueid):
    with connect(self.dbPath) as con:
        query ="""SELECT Publications.publicationInternalId, Publications.doi, Publications.title, 
        Publications.publicationYear, Venues.venueID, CitedPublications.id_all_references, 
        GROUP_CONCAT(Authors.authorID)
        FROM Publications
        LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN CitedPublications ON CitedPublications.publicationInternalId == Publications.publicationInternalId
        WHERE Venues.venueID LIKE '%{0}%'
        GROUP BY Publications.doi;""".format(venueid)
        pubinvenue = read_sql(query, con)
        pubinvenue= pubinvenue.rename(columns={"GROUP_CONCAT(Authors.authorID)": "all_authors_id"})
    return pubinvenue

For the following three methods, we had to use a similar approach.

For <code>getJournalArticleInIssue</code>, <code>getJournalArticleInVolume</code>, <code>getJournalArticleinJournal</code> we actually all had to retrieve the same columns: <i>Publications.publicationInternalId, Publications.doi, Publications.title,Publications.publicationYear, Venues.venueID, CitedPublications.id_all_references, GROUP_CONCAT(Authors.authorID), Publications.issue, Publications.volume</i>.

Basically we were working with the same columns as when working with a publication, but we had to keep in mind that the subclass <i>JournalArticle</i> had also volume and issue as attributes, so we added them in the <code>SELECT</code>.

What changed through these three different methods was the number of the input parameters, which we solved through using the usual <code>format</code> method and placeholders.

In the first method we had actually retrieve the specific <i>JournalArticle</i>, which had the specific volume, issue and venue id explicited in the parameter, whereas in the second we had in input just the volume and venue id and in the last one just the venue id.

In [ ]:
def getJournalArticlesInIssue(self,issue, volume,journal_id):
        with connect(self.dbPath) as con:
            query ="""SELECT Publications.publicationInternalId, Publications.doi, Publications.title, 
            Publications.publicationYear, Venues.venueID, CitedPublications.id_all_references, 
            GROUP_CONCAT(Authors.authorID), Publications.issue, Publications.volume
            FROM Publications
            LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN CitedPublications ON CitedPublications.publicationInternalId == Publications.publicationInternalId
            WHERE  Publications.issue = '{0}' AND Publications.volume = '{1}' AND Venues.venueID LIKE '%{2}%'
            GROUP BY Publications.publicationInternalId;""".format(issue,volume,journal_id)
            journalissue = read_sql(query, con)
            journalissue= journalissue.rename(columns={"GROUP_CONCAT(Authors.authorID)": "all_authors_id"})
        return journalissue
def getJournalArticlesInVolume(self,volume,journal_id):
    with connect(self.dbPath) as con:
        query ="""SELECT Publications.publicationInternalId, Publications.doi, Publications.title, 
        Publications.publicationYear, Venues.venueID, CitedPublications.id_all_references, 
        GROUP_CONCAT(Authors.authorID), Publications.issue, Publications.volume
        FROM Publications
        LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN CitedPublications ON CitedPublications.publicationInternalId == Publications.publicationInternalId
        WHERE Publications.volume = '{0}' AND Venues.venueID LIKE '%{1}%'
        GROUP BY Publications.publicationInternalId;""".format(volume,journal_id)
        journalvolume = read_sql(query, con)
        journalvolume= journalvolume.rename(columns={"GROUP_CONCAT(Authors.authorID)": "all_authors_id"})
    return journalvolume
def getJournalArticlesInJournal(self,journal_id):
    with connect(self.dbPath) as con:
        query ="""SELECT Publications.publicationInternalId, Publications.doi, Publications.title, 
        Publications.publicationYear, Venues.venueID, CitedPublications.id_all_references, 
        GROUP_CONCAT(Authors.authorID), Publications.issue, Publications.volume
        FROM Publications
        LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
        LEFT JOIN CitedPublications ON CitedPublications.publicationInternalId == Publications.publicationInternalId
        WHERE Venues.venueID LIKE '%{0}%'
        GROUP BY Publications.publicationInternalId;""".format(journal_id)
        journaljournal = read_sql(query, con)
        journaljournal= journaljournal.rename(columns={"GROUP_CONCAT(Authors.authorID)": "all_authors_id"})
    return journaljournal

One thing that we actually had to take in consideration with the following method - <code>getProceedingsByEvent</code> - and be careful about was the fact that the event in input could be just a <b>partial match</b>, so, in the <i>WHERE</i> condition, we added both the <i>[wildcards](https://www.w3schools.com/sql/sql_wildcards.asp)</i> character <code>%</code>, around the placeholders (which basically mean zero or more character before or after what we pass as input) and the <code>[LIKE](https://www.w3schools.com/sql/sql_like.asp)</code> operator.

In [ ]:
def getProceedingsByEvent(self,event):
        with connect(self.dbPath) as con:
            query ="""SELECT Venues.venueInternalId, Publications.publicationVenue, Venues.venueID, Publications.publisher, Publications.event
            FROM Publications
            LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN CitedPublications ON CitedPublications.publicationInternalId == Publications.publicationInternalId
            WHERE Publications.event LIKE '%{0}%'
            GROUP BY Publications.publicationInternalId;""".format(event)
            procevent= read_sql(query, con)
            procevent=procevent.rename(columns={"publicationVenue":"title", "publisher":"organizationID"})
        return procevent

For the method <code>getPublicationAuthors</code>, we used the same method as always but selecting the columns which characterizes the class <i>Author</i> (<i>Authors.familyName, Authors.givenName, Authors.authorID</i>).

We worked <code>FROM</code> the table <i>Authors</i> and joined it with the one of <i>Publications</i> to then execute the <code>WHERE</code> condition.

In [ ]:
def getPublicationAuthors(self,publid):
        with connect(self.dbPath) as con:
            query ="""SELECT Authors.familyName, Authors.givenName, Authors.authorID
            FROM Authors
            LEFT JOIN Publications ON Authors.publicationInternalId == Publications.publicationInternalId
            WHERE Publications.doi == '{0}';""".format(publid)
            publidf= read_sql(query, con)
        return publidf

For the following method, <code>getPublicationsByAuthorName</code> we actually brought together all the logic we had learned through the previous queries, since also in this case we had to use the <code>LIKE</code> operator and the <code>%</code> since we'd be dealing with a potential partial match in input.

We also used the same trick as for the code <code>getPublicationsByAuthorId</code> to obtain all the authors of the same publication in one row, merging a <code>DataFrame</code> obtained through a second query.

Finally we added the the concatenation operator <code>||</code> to concatenate together the family and given name, because we found that through trying the code, the original idea we had would work only if passed in input the family or the given name of the author (even just a partial match).

We wanted to actually make sure that also through inserting the full name (given and family, together) of the author (lowercase or uppercase and with a space between or not), it'd be returning all the results properly.

In [ ]:
def getPublicationsByAuthorName(self,authorname):
        import re
        with connect(self.dbPath) as con:
            query ="""SELECT Authors.familyName,
            Publications.publicationInternalId, Publications.doi, Publications.title, 
            Publications.publicationYear, Venues.venueID, CitedPublications.id_all_references, 
            GROUP_CONCAT(Authors.authorID), Publications.issue, Publications.volume
            FROM Publications
            LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN Venues ON Venues.publicationInternalId == Publications.publicationInternalId
            LEFT JOIN CitedPublications ON CitedPublications.publicationInternalId == Publications.publicationInternalId
            WHERE 
            Authors.givenName ||''|| Authors.familyName LIKE '%{0}%' OR
            Authors.givenName ||' '|| Authors.familyName LIKE '%{0}%' OR
            Authors.familyName ||''|| Authors.givenName LIKE '%{0}%' OR
            Authors.familyName ||' '|| Authors.givenName LIKE '%{0}%'
            GROUP BY Publications.publicationInternalId;""".format(authorname)

            authornamedf = read_sql(query, con)
            
          
            
            query_2="""SELECT  GROUP_CONCAT(Authors.authorID), Publications.publicationInternalId
            FROM PUBLICATIONS
            LEFT JOIN Authors ON Authors.publicationInternalId == Publications.publicationInternalId
            GROUP BY Publications.publicationInternalId;"""
            all_authors= read_sql(query_2, con)
            all_authors= all_authors.rename(columns={"GROUP_CONCAT(Authors.authorID)":"all_authors_id"})
            
            publications_by_author_name_df=authornamedf.merge(all_authors)
        return  publications_by_author_name_df

The last question resulted a bit tricky, since we'd have to return a DataFrame with all the <i>distinct</i> publishers that have published the publications specified in input (as a list).

To obtain this we actually created another <code>DataFrame</code> (at the moment empty), and then through a for iterated all the elements present in the input list.

* First we executed a normal query to obtain what we needed with the first element iterated, and as always stored it in a variable through <code>read_sql</code>, then we actually concatenated this <code>DataFrame</code>, with the empty one we created at the beginning.
* We then went back to the beginning with another element of the input list, and this time the empty <code>DataFrame</code> we had at the beginning would be merged with the new one, effectively.

This way we'd return the distinct publishers, as wanted by the method.

In [ ]:
def getDistinctPublisherOfPublications(self, plist):
        import pandas as pd
        from pandas import DataFrame
        from pandas import concat
        with connect(self.dbPath) as con:
            pubId=DataFrame()
            for el in plist:
                query="""SELECT Publishers.organizationID, Publishers.name
                FROM Publishers
                LEFT JOIN Publications ON Publications.publisher==Publishers.publisherInternalId
                WHERE Publications.doi ='{0}';""".format(el)
                distinctpub_df=read_sql(query, con)
                pubId=pd.concat([pubId, distinctpub_df])
            return pubId



---



# Triplestore Classes

## The Graph Database
 

Our goal was that of translating such a data model into an RDF graph database:
![image.png](attachment:55bd63c9-6721-452b-aec3-905deb8d53e3.png)

First of all, we have identified the names of all the most concrete classes (e.g. <b>JournalArticle</b>, <b>BookChapter</b>, <b>ProceedingsPaper</b>,<b> Journal</b>,<b> Book</b>,<b> Proceedings</b>,<b> Person</b>, <b>Organization</b>). </br>
Then we established that each attribute of each UML class would have been represented by a distinct RDF property which would have been involved in statements where the subjects are always resources of the class in consideration and the objects are simple literals (i.e. values). Of course, we had to identify the names of these properties (i.e. the URLs).</br>
We also decided that each relation starting from an UML class and ending in another UML class would have been represented by a distinct RDF property which would have been involved in statements where the subjects are always resources of the source class while the objects are resources of the target class. As well as said before, we had to identify the names of these properties (i.e. the URLs).



For creating the graph database, we have used the method `uploadData` of the class `TriplestoreQueryProcessor`.
 This method takes in input the path of the file you have to upload and returns a boolean which confirmes or not the effective success of the operation. <br>We have implemented this method starting from the creation of an empty `Graph` and of the `URIRef` for all the classes, attributes and relations among classes in our data model. For creating them, we have used the existing standards of [Schema.org](https://schema.org/). <br>

In [ ]:
from rdflib import Graph

my_graph = Graph()

from rdflib import URIRef
#classes of resources 
Person= URIRef("https://schema.org/Person")
JournalArticle= URIRef ("https://schema.org/ScholarlyArticle")
BookChapter=URIRef ("https://schema.org/Chapter")
Journal= URIRef("https://schema.org/Periodical")
Book= URIRef("https://schema.org/Book")
Organization= URIRef("https://schema.org/Organization")
ProceedingsPaper= URIRef("https://schema.org/Article")
Proceedings= URIRef("https://schema.org/Event")

#attributes related to classes
givenName = URIRef("https://schema.org/givenName")
familyName = URIRef("https://schema.org/familyName")
id = URIRef("https://schema.org/identifier")
publicationYear = URIRef("https://schema.org/datePublished")
title = URIRef("https://schema.org/name")
issue = URIRef("https://schema.org/issueNumber")
volume = URIRef("https://schema.org/volumeNumber")
name = URIRef("https://schema.org/name")
chapterNumber = URIRef("https://schema.org/Number")
event = URIRef("https://schema.org/releasedEvent")
n_citations = URIRef("http://purl.org/spar/cito/isCitedBy")

#relations among classes
publicationVenue = URIRef("https://schema.org/isPartOf")
publisher= URIRef("https://schema.org/publisher")
cites = URIRef ("https://schema.org/citation")
author = URIRef("https://schema.org/author")
creators = URIRef ("https://schema.org/creator")
all_citations = URIRef ("https://schema.org/relatedLink")



Then, we created our `base_url`, in order to use it as a starting point for generating the URI identifying our resources: 

In [ ]:
base_url = "https://my_little_py.github-io/res/"

Then, we had to upload our data on the database starting from two files: a CSV containing mostly the data regarding the attributes of the class `Publication` and a JSON containing the information about the `Authors`, `Venues`, `Publishers` and `References` to other publications.<br> We had to put these data on the database separately, in a way that , independently from the file format we were loading, we would have obtained a graph database.
So, we have checked the format of the files we had to upload: in order to distinguish between a JSON and a CSV, we have used an <b>if</b> statement. <br>After doing that, we wrote the code for loading our data on the database according to the file format they are stored into and to the structure of the JSON and CSV files we had to load.

In order to upload the data contained in the CSV file, we had first to open the CSV file. To do this, we used the function `read_csv` from Pandas, that takes in input a file path and returns a `DataFrame` representing such tabular data. </br>
We have also specifyed an additional input named parameter, i.e. `dtype`, which enables the specification of a dictionary where the keys are column names, while the values are the strings representing the data type for each column. Instead, to force Pandas to use an empty string as default for string-based column in case of missing values, it is enough to tell the function `read_csv` not to use the default `NaN` for missing value by setting the input named parameter `keep_default_na` to `False`.

In [ ]:
if path.endswith(".csv"):
    from pandas import read_csv
    df_graph_publications = read_csv (path,
                          keep_default_na=False,
                          encoding="utf-8",
                        dtype={
                       "id": "string",
                       "title": "string",
                       "type": "string",
                       "publication year": "int",
                       "issue": "string",
                       "volume": "string",
                       "chapter": "string",
                       "publication venue": "string",
                       "venue_type": "string",
                       "publisher":"string",
                       "event":"string"            
                    }) 
    

The next step was that of adding data to our graph database in the form of <b>RDF statements</b>. Each statement is a triple subject-predicate-object, where the subject is a resource, the predicate is a property, and the object is either a resource or a literal (i.e. a string). </br>
We needed to use the method `add` to add a new RDF statement to a graph. Such method takes in input a tuple of three elements defining the subject (an `URIRef`), the predicate (another `URIRef`) and the object (either an `URIRef` or a `Literal`) of the statements.
</br>

The following code shows how we populated the RDF graph using the data obtained by processing the CSV document. For instance, all the venues have been created using the following code:

In [ ]:
#venues
for idx, row in df_graph_publications.iterrows():
    local_id= "venue-" + str(row["id"])
    subj = URIRef(base_url + local_id)

    if row["venue_type"] == "journal":
        my_graph.add((subj, RDF.type, Journal))

    if row["venue_type"] == "book":
        my_graph.add((subj, RDF.type, Book))

    if row["venue_type"] == "proceedings":
        my_graph.add((subj, RDF.type, Proceedings))
        my_graph.add((subj, event, Literal(row["event"])))

    my_graph.add((subj, title, Literal(row["publication_venue"])))
    my_graph.add((subj, publisher, URIRef(base_url + "publisher-" + str(row["publisher"]))))

The same approach has been used to add information about the publications, as shown as follows:

In [ ]:
#publications
            
for idx, row in df_graph_publications.iterrows():
    local_id= "publication-" + str(row["id"])

    subj= URIRef(base_url + local_id)

    if row["type"] == "journal-article":
        my_graph.add((subj, RDF.type, JournalArticle))
        my_graph.add((subj, issue, Literal(row["issue"])))
        my_graph.add((subj, volume, Literal(row["volume"])))

    if row["type"] == "book-chapter":
        my_graph.add((subj, RDF.type, BookChapter))
        my_graph.add((subj, chapterNumber, Literal(row["chapter"])))

    if row["type"] == "proceedings-paper":
        my_graph.add((subj, RDF.type, ProceedingsPaper))


    my_graph.add((subj, title, Literal(str(row["title"]))))
    my_graph.add((subj, id, Literal(row["id"])))
    my_graph.add((subj, publicationYear, Literal(str(row["publication_year"]))))

In this way we added to our RDF graph the data that we could obtain from the CSV file. </br>
The difficult part has been that of connecting this information with all the other data contained in the JSON file. We managed this task using the `local_id`, that we created (for each publication), by adding to the string "publication-" the value contained in the column "id" of the `DataFrame`, that is the doi, a unique value for each publication.

In order to upload the data contained in the JSON file, we had first to use specific functions of the Python package `json` to load the JSON document in Python. In particular, we used the function `load` to import in Python a JSON object, that must be imported from the json package as usual. In this way in our `graph_data` variable we obtained a list of dictionaries, that has been created to map the JSON array.

In [ ]:
def uploadData (path):
    if path.endswith(".json"):
        from json import load 
        with open(path, "r", encoding="utf-8") as f:
            graph_data = load(f)

Then we have created some dataframes starting from the key-value pairs of the dictionaries contained in the list accesible with the variable `graph_data`. We have chosen this way for accessing the information we needed for creating the triple statements. For example, starting from the key `authors`, we have created a `DataFrame` with the information about the authors (<b>givenName</b>, <b>familyName</b>, <b>orcid</b>), `df_author`, with the aim of creating the statements having the authors as subject. 

In [ ]:
import pandas as pd
from pandas import DataFrame
#authors' dataframe

findex=0
for x in graph_data["authors"]:
    for y in graph_data["authors"][x]:
        findex+=1

df_author=pd.DataFrame(columns=["family","given","orcid"],index=range(findex))#dataframe with authors information
ind=0
for x in graph_data["authors"]:
    for y in graph_data["authors"][x]:
        df_author.iloc[ind] = (y["family"],y["given"],y["orcid"])
        ind+=1

So we have iterated on its rows (one row for each author) for creating our `local_id` for the authors.<br>
We have also created a dictionary for storing all the URIs as values of keys containing the `orcid`, in order to use them in the relations not having the authors as subject and in which the class `author` is the target one.<br>
And within this iteration, we have added the statements as we did for the data coming from the CSV document.

In [ ]:
author_internal_id = {}
for idx, row in df_author.iterrows():
    local_id ="author-" + str(row["orcid"])

    subj=URIRef (base_url + local_id)
    
    # We put the new author resources created here, to use them
    # when creating publications
    author_internal_id[row["orcid"]] = subj
    author_fullname=(str(row["given"])+ ""+ str(row["family"]))

    my_graph.add((subj, RDF.type, Person))
    my_graph.add((subj, givenName, Literal(row["given"])))
    my_graph.add((subj, familyName, Literal(row["family"])))
    my_graph.add((subj, name, Literal(author_fullname)))
    my_graph.add((subj, id, Literal(row["orcid"])))

We have also created another dataframe, `df_pub_author`, with the `orcid` of the authors and the `doi` of the publications, and we have used it later, together with the dictionary `author_internal_id`, in order to access to the URIs to assign them as object of the <b>relation between Publications and Authors</b>, as shown here:


In [ ]:
#dataframe with id publications and authors' orcid 
df_pub_author = pd.DataFrame (columns=["id", "orcid"], index=range(findex)) #dataframe with doi and orcid of the authors
ind =0
for x in graph_data["authors"]:
    for y in graph_data["authors"][x]:
        df_pub_author.iloc[ind] = (x,y["orcid"])
        ind+=1

#publications dataframe
findex=0
for x in graph_data["authors"]:
    findex += 1
df_doi = pd.DataFrame (columns= ["id"], index = range(findex))
ind = 0
for x in graph_data["authors"]:
    df_doi.iloc[ind] = (x)
    ind +=1
    
#statements with publications as subject

for idx, row in df_doi.iterrows():
    local_id = "publication-" + str(row["id"])
    subj = URIRef (base_url + local_id)

    #predicate author
    for ind, ref in df_pub_author.iterrows():
        if ref["id"] == row["id"]:
            my_graph.add((subj, author, author_internal_id[ref["orcid"]]))

In the code above it is possible to see how we handled the <b>statements with Publications as subjects</b> coming from the JSON file. We used as starting point the doi contained as keys in the authors' dictionary stored in the variable `graph_data` and we created a `DataFrame` with just the publications' doi, called `df_doi`. Iterating over this dataframe we created the `local_id` of our resources as said before, so adding to the string "publication-" the value contained in the column "id" of the DataFrame, that is the doi, a unique value for each publication. In this way we succeeded in linking data about publications coming from the CSV document with the ones coming from the JSON. 

We used the same approach for all the other classes. So we created:
* a dataframe with publishers' information;
* a dataframe with publications' doi and venues' identifiers;
* a dataframe with publications' doi and references' doi

In [ ]:
#dataframe for publishers 
findex = 0
for x in graph_data["publishers"]:
    findex+=1
df_publishers = pd.DataFrame(columns=["id", "name"], index=range(findex))
ind=0 
for x in graph_data["publishers"]:
    df_publishers.iloc[ind] = (graph_data["publishers"][x]["id"], graph_data["publishers"][x]["name"])
    ind +=1
#statements with publishers as subject
for idx, row in df_publishers.iterrows():
    local_id = "publisher-" + str(row["id"])

    subj= URIRef (base_url + local_id)

    my_graph.add((subj, RDF.type, Organization))
    my_graph.add((subj, name, Literal(row["name"])))
    my_graph.add((subj, id, Literal(row["id"])))

We managed the link between the class `Venue` and the class `Organization` again with the `local_id`, that this time we created using the string "publisher-" and the identifier of each organization, that is both in the JSON and in the CSV (in the column `"publisher"`).

In [ ]:
#dataframe with publications'doi and venues id
findex = 0
for x in graph_data["venues_id"]:
    findex += 1
df_pub_venues = pd.DataFrame(columns = ["doi","venues_id"], index= range(findex))
ind =0
for x in graph_data["venues_id"]:
    df_pub_venues.iloc[ind] = (x, graph_data["venues_id"][x])
    ind +=1

#statements with venues as subject
venue_internal_id = dict()
for idx, row in df_pub_venues.iterrows():

    local_id = "venue-" + str(row["doi"])
    subj = URIRef (base_url + local_id)
    
    # We put the new venues resources created here, to use them
    # when creating the relation between publications and venues 
    venue_internal_id[str(row["venues_id"])] = subj

    my_graph.add((subj, id, Literal(row["venues_id"])))


We have used the dictionary `venue_internal_id`, in order to access to the URIs to assign them as object of the <b>relation between Publications and Venues</b>, as shown here:


In [ ]:
for idx, row in df_doi.iterrows():
    local_id = "publication-" + str(row["id"])
    subj = URIRef (base_url + local_id)

    #predicate isPartOf
    for ind, ref in df_pub_venues.iterrows():
        if ref["doi"] == row["id"]:
            my_graph.add((subj, publicationVenue, venue_internal_id[str(ref["venues_id"])]))

In [ ]:
#references 
findex=0
for x in graph_data["references"]:
    for y in graph_data["references"][x]:
        findex+=1
df_references = pd.DataFrame(columns=["id", "id_references"], index=range(findex))
ind = 0
for x in graph_data["references"]:
    for y in graph_data["references"][x]:
        df_references.iloc[ind]= (x, y)
        ind += 1

publication_internal_id= {}
            
for idx, row in df_doi.iterrows():

    local_id = "publication-" + str(row["id"])
    subj = URIRef (base_url + local_id)
    
    #we put the resources just created in this dictonary to use 
    #them in the statements with predicate "cites"
    publication_internal_id[row["id"]] = subj
    
for idx, row in df_doi.iterrows():
    local_id = "publication-" + str(row["id"])
    subj = URIRef (base_url + local_id)

    #predicate cites
    for ind, ref in df_references.iterrows():
        if ref["id"] == row["id"]:
            if ref["id_references"] in publication_internal_id:
                my_graph.add((subj, cites, publication_internal_id[ref["id_references"]]))

References are publications so in this case we stored the publications in a dictionary, using as keys the publications' doi, and we used the values as objects in the statements with the predicate "schema:citation".

Some clarifications: 
* we created two other dataframes, `df_all_authors_id` and `df_all_references`, containing respectively the doi of the publication and the orcid of all the authors who wrote that publication, and the doi of the publication and the doi of the publications cited by the publication that we are considering. We used these two dataframes to add, through new statements (one with as subject the class Publication and as predicate "schema:creator", and the other with as subject the class `Publication` and as predicate "schema:relatedLink"), all the authors of a publication and all the publications cited by one publication in a single cell.
* we created a dictionary with publications' doi as keys and number of times a publication is cited as values and we used these values as objects in the statements having the class  `Publication ` as subject and schema:isCitedBy as predicate. We added this information in our graph database in order to use it with some methods of the `TriplestoreQueryProcessor `, in particular  `getMostCitedPublication ` and  `getMostCitedVenue `.

## The Triplestore Query Processor

Then, we created the `TriplestoreQueryProcessor`, which according to the data model is a subclass of the `TriplestoreProcessor` and inherits its attribute `endpointUrl`.

In [ ]:
class TriplestoreQueryProcessor (TriplestoreProcessor):
    def __init__(self):
        super().__init__(self.endpointUrl)

We created all the methods contained in the data model for this class, adding in the parameters for each method the input value to be used for the query (for example, in `getPublicationsPublishedInYear`, we put `year` as a parameter), defining them in this way:

In [ ]:
def getPublicationsPublishedInYear (self, year):

After that, we defined a variable for the query corresponding to the method. We have stored each query as a list of strings. This was done in order to insert within the SPARQL query string the variable corresponding to the input value used for searching information in the database. 

In [ ]:
    query_1= [ """
            PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX schema: <https://schema.org/>

            SELECT ?internalID ?id ?type ?title ?chapterNumber ?issue ?volume ?publicationVenue ?author ?cites
            WHERE {
                VALUES ?type {
                    schema:ScholarlyArticle schema:Chapter schema:Article}
                ?internalID rdf:type ?type .
                ?internalID schema:identifier ?id.
                ?internalID schema:name ?title.
                ?internalID schema:creator ?author.
                ?internalID schema:datePublished""", str("'"+year+"'"), ".", 
                """?internalID schema:isPartOf ?publicationVenue.
                                    
            
                OPTIONAL { ?internalID schema:Number ?chapterNumber .}
                OPTIONAL {?internalID schema:issueNumber ?issue .}
                OPTIONAL {?internalID schema:volumeNumber ?volume .}
                OPTIONAL {?internalID schema:relatedLink ?cites.}
                
            }
            """
                ]

The queries are written in the SPARQLQuery language for graph databases and follow its syntax. They are composed by:
 * the prefixes, indicating the URLs of the schemas we have used for getting our properties (RDF and Schema.org);
 * the `SELECT` part, in which we selected all the variables we need in the result of the query:
 * the `WHERE` close, which contains the statements to be satisfied in the query;
 * eventually,some `OPTIONAL` statements that don't have to be necessarily satisfied.<br>
 
 For realizing some of the queries we have used also other `SPARQL` functions(?).<br>
 For example, we have used the `FILTER` together with `REGEX` when we needed to check if the input strings was contained in the data of our database. This happened, for example, in the `getPublicationsByAuthorsName` query. For creating this specific query, we have added a triple statement in the graph database. This statement has the author resource as `subject`, the property `name` from Schema.org as a `predicate` and a Literal with the author's fullname as `object`.
 Thanks to this statement we were able to check if th input string was contained in either in the given or in the family name(or in both) of a specific author, in this way: 


In [ ]:
    def getPublicationsByAuthorName(self, inputstring):
            
            query_10=["""
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX schema: <https://schema.org/>
    SELECT ?internalID ?title ?doi ?publicationYear ?venueID ?type ?issue ?volume ?chapterNumber ?all_authors_id ?id_all_references
        WHERE{ 
        ?internalID schema:author ?author;
                    schema:name ?title;
                    schema:datePublished ?publicationYear;
                    schema:identifier ?doi;
                    rdf:type ?type.
        OPTIONAL{?internalID schema:isPartOf ?publicationVenue.
        ?publicationVenue schema:identifier ?venueID.}
                                
        
        ?author  schema:name ?fullname.
        ?author  schema:identifier ?authorID.
        
        ?internalID schema:creator ?all_authors_id.         
        FILTER REGEX(?fullname,""",str("'"+inputstring+"'"), """, "i").
            
        OPTIONAL {?internalID schema:issueNumber ?issue.
                ?internalID schema:volumeNumber ?volume.
                }
        OPTIONAL {?internalID schema:Number ?chapterNumber.}
        OPTIONAL {?internalID schema:relatedLink ?id_all_references.}
        }
    """]

After having written the queries in this way, we needed to join all the strings composing each single query in a unique string, in order to use it in the following step. For doing that, we used the `.join` method, which allowed to transform the list of strings into one single string, in this way:

In [ ]:
    stringa=(" ".join(query_1))

At this point, we had to obtain a DataFrame as the final result of every qwery. Therefore, we have used the funcion `get`imported from `sparql_dataframe`. This funcion, taking in input the endpoint URL of the database and the string representing the query to do in the database, as well as the boolean `True`, allowed us obtaining a dataframes as a result from every query.

In [ ]:
    from sparql_dataframe import get
    
    df_final = get (TriplestoreProcessor.getEndpointUrl(TriplestoreProcessor), stringa, True)

Then, we have modified every dataframe setting the type of the columns as `string` and remuving the final ".0" from  string values still containing it. Indeed, launching the queries, we noticed that some columns of the final dataframes had not the type `string` even if we had set it in the initial dataframe created while opening the CSV. We have also tried to add them as string Literals in the statement of the graph, but the problem was still present. Therefore we have used a function in each method for removing the ".0" from the columns of the dataframe resulting from the query.

In [ ]:
     
    def remove_dotzero(s):
        return s.replace(".0", "")

# The Generic Query Processor

In creating the Generic query processor, we have faced some issues. The first thing we did was defining the class.
We have also defined here the variable `queryProcessor` , which is the list containing the two query processors to use for the generic queries.

In [ ]:
class GenericQueryProcessor(object):
    def __init__(self, queryProcessor):
        self.queryProcessor=queryProcessor


The first two methods we defined are  `addQueryProcessor`, meant to add query processors to the `queryProcessor` list, and `cleanQueryProcessor`, meant to remove from the same list the query processors we want to use.


In [ ]:
def cleanQueryProcessors(self):
        for obj in self.queryProcessor:
            self.queryProcessor.remove(obj)
            return self.queryProcessor
            
def addQueryProcessor(self, input):
        self.queryProcessor.append(input)

Then, for the methods involving the execution of the queries, we have proceeded thid way:
* we have defined the method using the input as a parameter;
* we have created an empty dataframe with the aim of concatenating it to the dataframes resulting from the separated queries;
* then, supposing that we have the two processors in the list `queryProcessor`, we have iterated on that list for executing the specific query of the method on every processor;
* for every query executed this way, we have used the `concat` function of `pandas` for joining the resulting dataframe with the `result` dataframe, initially set as empty.

After having obtained a unique dataframe with the results of the execution of the query in every processor added in the list, we have applied some modifications to this dataframe: we have removed the the `NaN` from the empty fields and we have dropped the duplicates. We have proceeded removing the duplicates because we wanted to create, starting from this dataframe, a single object for each row.

In [ ]:
def getPublicationsPublishedInYear(self, inputYear):
        result = DataFrame()
        self.finalresultlist=[]
        for processor in self.queryProcessor: 
            q=processor.getPublicationsPublishedInYear(inputYear)
            result = concat([result, q],ignore_index=True)
        #removing the NaN
        result= result.fillna("")
        result.drop_duplicates(subset="doi", keep = "first", inplace=True)
                    
        

Then, we went on with the instantiation of the objects as requested for the implementation of these methods. 
Our aim was to use each row for instantiating an object of the corresponding class in the data model.
For doing it, we have iterated on the row of the `result` dataframe, in order to get from its columns the constructors needed for the instantiation. <br>

In this step, we had to face a problem: in the case of some parameters, like `author` for the class `Publication`, we had more than one corresponding value (e.g. more than one author for a publication). How to instanciate a `Publication` containing all of them in the values of the parameter `author`? <br>
At first, we tried creating a list containing all the unique identifiers for the authors of the same publications, with the aim of using this list as a value for the parameter author, but we had problems in implementing this idea.<br>
Therefore, we thought that a solution could be grouping all the identifiers of the authors of a single publication into one single value and using it for instantiating the `Publication` objects. <br>

This implied the modification of our databases: we have added a statement in the graph database containing as object all the identifiers for the authors of a publication and a dataframe in the relational database containing, the same way, all the identifiers for all the authors of every publication.<br>
This allowed us to select these values in the queries and use them for instantiating the objects of that class.


In [ ]:
        for row_idx,row in result.iterrows():
                    x=Publication(identifiers=row["doi"],title=row["title"],publicationYear=row["publicationYear"],
                    publicationVenue=row["venueID"], author = row["all_authors_id"],
                                citedpublication = row["id_all_references"])  

                    self.finalresultlist.append(x)
                    
        return self.finalresultlist